In [1]:
import sys
sys.path.append('../xrun')

In [2]:
import re

from pathlib import Path

import numpy as np
import pandas as pd

from IPython.core.display import HTML
from sklearn.metrics import pairwise_distances

from xrun.data.run_info import RunInfo

In [3]:
def load_cost_from_file(file_path: Path):
    if file_path.exists():
            with open(file_path, "r") as f:
                return float(f.read())
    else:
        print(f"Warning: {file_path} not found!")
    return None

def get_costs(file_paths):
    costs = []
    for index, run_file_path in enumerate(file_paths):
        run_info = RunInfo.load_json(run_file_path)
        real_cost = load_cost_from_file(run_file_path.parent / "real_cost.txt")
        coreset_cost = load_cost_from_file(run_file_path.parent / "coreset_cost.txt")
        costs.append({
            "dataset": run_info.dataset,
            "algorithm": run_info.algorithm,
            "k": run_info.k,
            "running_time": int(run_info.duration_secs),
            "real_cost": real_cost,
            "coreset_cost": coreset_cost,
            "run_file_path": str(run_file_path),
        })
    return pd.DataFrame(costs)

def aggregate_costs(df_costs: pd.DataFrame):
    group_columns = ["dataset", "algorithm", "k"]
    df_top_k = df_costs.groupby(group_columns, as_index=False).head(10)
    df_aggr_costs = df_top_k.groupby(group_columns, as_index=False).agg(
        experiment_count=("algorithm", "count"),
        real_cost_mean=("real_cost", "mean"),
        real_cost_std=("real_cost", "std"),
        coreset_cost_mean=("coreset_cost", "mean"),
        coreset_cost_std=("coreset_cost", "std"),
        running_time_mean=("running_time", "mean"),
        running_time_std=("running_time", "std"),
    )
    df_aggr_costs["ratio1"] = df_aggr_costs["real_cost_mean"] / df_aggr_costs["coreset_cost_mean"]
    df_aggr_costs["ratio2"] = df_aggr_costs["coreset_cost_mean"] / df_aggr_costs["real_cost_mean"]
    df_aggr_costs["distortion"] = df_aggr_costs[["ratio1", "ratio2"]].max(axis=1)
    df_aggr_costs["running_time"] = pd.to_timedelta(df_aggr_costs.running_time_mean, unit='s')
    df_aggr_costs["running_time_formatted"] = df_aggr_costs["running_time"].map(
        lambda x: f"{x.components.hours:02d}h {x.components.minutes:02d}m {x.components.seconds:02d}s"
    )
    return df_aggr_costs


def display_results_for(df_aggr_costs: pd.DataFrame, dataset_name: str, show_counts: bool=True, show_running_times:bool=True, show_costs: bool=True):
    pd.set_option('display.float_format', '{:.2e}'.format)
    
    display(HTML(f"<h2 style='border-bottom:solid 1px Black;padding-bottom:5px;'>Results for {dataset_name}</h2>"))
    df_filtered = df_aggr_costs[df_aggr_costs.dataset == dataset_name]
    
    html_str = ""
    
    if show_counts:
        df_counts = pd.pivot_table(df_filtered, values="experiment_count", index=["algorithm"], columns=["k"])
        df_counts = df_counts.rename_axis(None, axis=0).rename_axis(None, axis=1)
        html_str += f'<div style="border:solid 1px White; width:300px;float:left;">'
        html_str += f"<h4 style='text-align:center;'>Experiment Counts</h4>"
        html_str += df_counts.style.format(precision=0).to_html()
        html_str += "</div>"
    
    if show_running_times:
        df_run_times = pd.pivot_table(df_filtered, values="running_time_formatted", index=["algorithm"], columns=["k"], aggfunc=lambda x: x)
        df_run_times = df_run_times.rename_axis(None, axis=0).rename_axis(None, axis=1)

        html_str += f'<div style="border:solid 1px White; float:left;">'
        html_str += f"<h4 style='text-align:center;'>Average Running Times</h4>"
        html_str += df_run_times.to_html()
        html_str += "</div>"
        
    display(HTML(html_str))

    if show_costs:
        df_real_costs = pd.pivot_table(df_filtered, values="real_cost_mean", index=["algorithm"], columns=["k"])
        df_real_costs = df_real_costs.rename_axis(None, axis=0).rename_axis(None, axis=1)
        
        df_coreset_costs = pd.pivot_table(df_filtered, values="coreset_cost_mean", index=["algorithm"], columns=["k"])
        df_coreset_costs = df_coreset_costs.rename_axis(None, axis=0).rename_axis(None, axis=1)
        
        html_str = ""
        html_str += f'<div style="border:solid 1px #eee; float:left;">'
        html_str += f"<h4 style='text-align:center;'>Real costs</h4>"
        html_str += df_real_costs.to_html()
        html_str += "</div>"
        
        html_str += f'<div style="border:solid 1px #eee; float:left;">'
        html_str += f"<h4 style='text-align:center;'>Coreset costs</h4>"
        html_str += df_coreset_costs.to_html()
        html_str += "</div>"
        display(HTML(html_str))
    
    display(HTML(f'<h4>Distortions</h4>'))
    df_distortions = pd.pivot_table(df_filtered, values="distortion", index=["algorithm"], columns=["k"])
    df_distortions = df_distortions.rename_axis(None, axis=0).rename_axis(None, axis=1)
    display(df_distortions)


In [4]:
data_results_dir = Path("../data/odin-results/")
run_files = list(data_results_dir.glob("**/*.json"))

In [5]:
run_files.extend(list(Path("../data/cary-results/hardinstanceb1").glob("**/*.json")))
run_files.extend(list(Path("../data/cary-results/hardinstanceb2").glob("**/*.json")))

In [6]:
len(run_files)

2378

In [7]:
df_cost_data = get_costs(run_files)

In [8]:
df_aggr_costs = aggregate_costs(df_cost_data)

In [9]:
for dataset_name in df_aggr_costs.dataset.unique():
    display_results_for(df_aggr_costs, dataset_name=dataset_name, show_counts=True, show_running_times=True, show_costs=False)

,10,20,30,40,50
basic-clustering,10,10,10,10,10
bico,10,10,10,10,10
group-sampling,10,10,10,10,10
sensitivity-sampling,10,10,10,10,10
,10,20,30,40,50
basic-clustering,04h 28m 16s,08h 45m 38s,13h 44m 58s,18h 03m 35s,22h 16m 09s
bico,00h 00m 59s,00h 01m 05s,00h 01m 21s,00h 01m 25s,00h 01m 34s
group-sampling,01h 02m 55s,02h 16m 18s,03h 36m 25s,06h 29m 10s,08h 49m 38s
sensitivity-sampling,00h 58m 50s,02h 48m 01s,04h 44m 00s,05h 49m 38s,07h 43m 34s


,10,20,30,40,50
basic-clustering,1.22e+00,1.25e+00,1.27e+00,1.28e+00,1.29e+00
bico,1.64e+00,1.73e+00,1.77e+00,1.83e+00,1.87e+00
group-sampling,1.02e+00,1.02e+00,1.02e+00,1.02e+00,1.02e+00
sensitivity-sampling,1.00e+00,1.01e+00,1.01e+00,1.00e+00,1.00e+00


,10,20,30,40,50
basic-clustering,10,10,10,10,10
bico,10,10,10,10,10
group-sampling,10,10,10,10,10
sensitivity-sampling,10,10,10,10,10
,10,20,30,40,50
basic-clustering,04h 29m 11s,08h 50m 35s,12h 55m 09s,17h 17m 33s,21h 15m 06s
bico,00h 01m 23s,00h 01m 45s,00h 01m 47s,00h 01m 50s,00h 01m 59s
group-sampling,00h 54m 10s,02h 02m 02s,03h 05m 17s,05h 07m 10s,09h 11m 14s
sensitivity-sampling,01h 09m 39s,01h 58m 47s,04h 14m 06s,05h 35m 35s,08h 35m 06s


,10,20,30,40,50
basic-clustering,1.05e+00,1.14e+00,1.21e+00,1.26e+00,1.29e+00
bico,1.18e+00,1.44e+00,1.64e+00,1.79e+00,1.85e+00
group-sampling,1.01e+00,1.02e+00,1.02e+00,1.02e+00,1.02e+00
sensitivity-sampling,1.00e+00,1.00e+00,1.01e+00,1.00e+00,1.00e+00


,10,20,30,40,50
basic-clustering,10,10,10,10,10
bico,10,10,10,10,10
group-sampling,10,10,10,10,10
sensitivity-sampling,10,10,10,10,10
,10,20,30,40,50
basic-clustering,00h 51m 58s,01h 43m 29s,02h 26m 10s,03h 16m 30s,04h 08m 14s
bico,00h 00m 09s,00h 00m 10s,00h 00m 12s,00h 00m 14s,00h 00m 16s
group-sampling,00h 40m 52s,01h 26m 58s,02h 22m 44s,03h 13m 22s,04h 34m 37s
sensitivity-sampling,00h 42m 37s,01h 29m 04s,02h 42m 00s,03h 31m 07s,04h 24m 20s


...

,10,20,30,40,50
basic-clustering,2.50e+00,2.81e+00,2.42e+00,2.86e+00,NaN
bico,3.81e+00,3.80e+00,3.31e+00,4.00e+00,4.35e+00
group-sampling,1.01e+00,1.01e+00,1.01e+00,1.01e+00,1.01e+00
sensitivity-sampling,1.00e+00,1.01e+00,1.00e+00,1.00e+00,1.00e+00


,10,20,30,40,50
basic-clustering,8,8,8,5,nan
bico,10,10,10,10,10
group-sampling,10,10,10,10,10
sensitivity-sampling,10,10,10,10,10
,10,20,30,40,50
basic-clustering,01h 36m 52s,16h 21m 28s,07h 13m 48s,16h 40m 30s,NaN
bico,00h 00m 13s,00h 01m 15s,00h 00m 36s,00h 01m 40s,00h 04m 15s
group-sampling,00h 02m 28s,00h 22m 23s,00h 09m 46s,00h 50m 29s,03h 05m 48s
sensitivity-sampling,00h 02m 23s,00h 21m 04s,00h 09m 13s,00h 46m 34s,02h 52m 29s


,10,20,30,40,50
basic-clustering,1.06e+00,1.20e+00,1.22e+00,1.22e+00,NaN
bico,1.11e+00,1.27e+00,1.25e+00,1.24e+00,1.23e+00
group-sampling,1.01e+00,1.01e+00,1.01e+00,1.01e+00,1.01e+00
sensitivity-sampling,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00


,20,40,60,80,100
basic-clustering,10,10,10,10,10
bico,10,10,10,10,10
group-sampling,10,10,10,10,10
sensitivity-sampling,10,10,10,10,10
,20,40,60,80,100
basic-clustering,02h 00m 55s,03h 54m 17s,05h 42m 29s,07h 37m 53s,09h 41m 00s
bico,00h 00m 09s,00h 00m 09s,00h 00m 09s,00h 00m 10s,00h 00m 11s
group-sampling,00h 43m 48s,01h 33m 15s,02h 12m 40s,02h 51m 57s,03h 48m 28s
sensitivity-sampling,00h 41m 47s,01h 26m 52s,02h 08m 31s,02h 39m 58s,03h 00m 39s


,20,40,60,80,100
basic-clustering,1.02e+00,1.02e+00,1.01e+00,1.01e+00,1.01e+00
bico,1.06e+00,1.06e+00,1.06e+00,1.05e+00,1.04e+00
group-sampling,1.01e+00,1.02e+00,1.03e+00,1.03e+00,1.03e+00
sensitivity-sampling,1.01e+00,1.01e+00,1.01e+00,1.01e+00,1.00e+00
